In [1]:
import pandas as pd

df= pd.read_csv('Data/baltimore_crime_counts_2023.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'Data/baltimore_crime_counts_2023.csv'

In [ ]:
crimes = ['LARCENY','LARCENY FROM AUTO', 'AUTO THEFT','SHOOTING','ROBBERY - CARJACKING']
crimes_ts = df[crimes].copy()
crimes_ts.head()

In [ ]:
crimes_ts.isna().sum()

In [ ]:
## fill null values with 0
crimes_ts = crimes_ts.fillna(0)
crimes_ts

In [ ]:
crimes_ts.plot(grid=True,alpha=0.7);

In [ ]:
# IDENTIFYING SEASONALITY

plot_df = crimes_ts.loc['2017':'2021']
plot_df.plot(grid=True);

In [ ]:
plot_df = crimes_ts.loc['2017':'2021']
plot_df.plot(grid=True);

In [ ]:
plot_df.plot(subplots=True,figsize=(10,12),grid=True);

In [ ]:
plot_df_W = crimes_ts.loc['2017':'2021'].resample('W').sum()
plot_df_W.plot(grid=True,subplots=True,figsize=(10,15));

In [ ]:
plot_df_W.rolling(4).mean().dropna().plot(subplots=True,figsize=(10,12));

In [ ]:
# SEASONAL DECOMPOSITION

# import the statsmodels time series analysis api module
import statsmodels.tsa.api as tsa

In [ ]:
crime_type = "LARCENY FROM AUTO"
ts = plot_df_W[crime_type]
ts.plot(title=crime_type)

In [ ]:
decomp = tsa.seasonal_decompose(ts)
decomp

In [ ]:
# VISUALIZING DECOMP RESULTS

fig = decomp.plot()

In [ ]:
fig = decomp.plot()
fig.set_size_inches(10,8)
fig.tight_layout()

In [ ]:
decomp.trend

In [ ]:
decomp.trend.plot(title='Trend');

In [ ]:
decomp.seasonal.head()

In [ ]:
decomp.seasonal.plot(title='Seasonal');

In [ ]:
decomp.resid.head()

In [ ]:
decomp.resid.plot(title='Residual');

In [ ]:
# Adding decomposition components together
ts_model = decomp.trend + decomp.seasonal + decomp.resid
# Visualize the result
ax = ts_model.plot(label='Recombined from Decomposition')
ax.legend()

In [ ]:
ax = ts.plot(label='Original, Raw')
ts_model.plot(label='Recombined from Decomposition',ax=ax, ls='--', alpha=0.8)
ax.legend();

In [ ]:
seasonal = decomp.seasonal
ax = seasonal.plot(title=f'Seasonal Component for {crime_type}');
ax.grid(which='both', axis='x')

In [ ]:
import matplotlib.dates as mdates
minor_loc = mdates.MonthLocator()
fig, ax = plt.subplots()
ax.plot(seasonal)
ax.set(title=f'Seasonal Component for {crime_type}');
ax.xaxis.set_minor_locator(minor_loc)
ax.grid(which='major', axis='x',lw=1, color='k')
ax.grid(which='minor',axis='x',lw=0.5)
fig.autofmt_xdate(rotation=90, ha='center')

In [ ]:
seasonal.max()

In [ ]:
seasonal.min()

In [ ]:
magnitude = seasonal.max() - seasonal.min()
magnitude

In [ ]:
seasonal.idxmax()

In [ ]:
seasonal.idxmin()

In [ ]:
# ANNOTATING PERIODS

import matplotlib.dates as mdates
def plot_seasonal(seasonal):
    minor_loc = mdates.MonthLocator()
    fig, ax = plt.subplots()
    ax.plot(seasonal)
    ax.set(title=f'Seasonal Component for {crime_type}');
    ax.xaxis.set_minor_locator(minor_loc)
    ax.grid(which='major', axis='x',lw=1, color='k')
    ax.grid(which='minor',axis='x',lw=0.5)
    fig.autofmt_xdate(rotation=90, ha='center')
    
    return fig, ax

In [ ]:
fig, ax = plot_seasonal(seasonal)
ax.axvspan(seasonal.idxmin(),seasonal.idxmax(), color='lightgreen');

In [ ]:
delta = seasonal.idxmax() - seasonal.idxmin()
delta

In [ ]:
from scipy.signal import find_peaks
peaks, props = find_peaks(seasonal, height=seasonal.max())

In [ ]:
peaks

In [ ]:
props

In [ ]:
peak_dates = seasonal.index[peaks]
peak_dates

In [ ]:
fig,ax = plot_seasonal(seasonal)
ax.scatter(peak_dates, props['peak_heights'], color='orange');

In [ ]:
peak_dates

In [ ]:
period = peak_dates[1] - peak_dates[0]
period

In [ ]:
periods = []
for i in range(len(peak_dates)-1):
    delta = peak_dates[i+1] - peak_dates[i]
    periods.append(delta)
periods

In [ ]:
# FINAL ANNOTATION
fig, ax = plot_seasonal(seasonal)
season_start = seasonal.idxmin()
ax.axvspan( season_start, season_start +period, color='lightgreen')
ax.axvline(season_start, color='green', ls='--',
           label=f"Start of Season {season_start.strftime('%B')}")
ax.legend();

In [ ]:
seasonal.max() - seasonal.min()